In [29]:
import os
import sys
import re
import math

In [16]:
log_file = 'job.log'

In [17]:
if not os.path.isfile(log_file) or not os.path.exists(log_file):
    print("Error: Log file not found.")

In [18]:
idsmap = dict()
tx = dict()
rx = dict()
rssi = dict()

# regular expressions to match
regex_id = re.compile(r".* (?P<node_id>\d+).firefly < b'Rime configured with address (?P<addr>\w\w:\w\w).*\n")
regex_tx = re.compile(r".*TX (?P<addr>\w\w:\w\w).*\n")
regex_rx = re.compile(r".*RX (?P<addr_from>\w\w:\w\w)->(?P<addr_to>\w\w:\w\w), RSSI = (?P<rssi>-?\d+)dBm.*\n")

In [19]:
with open(log_file, 'r') as f:
    for line in f:

        # match address strings to map them to IDs
        m = regex_id.match(line)
        if m:
            # get dictionary of matched groups
            d = m.groupdict()

            # add to the dictionary
            idsmap[d['addr']] = d['node_id']
            continue
        
        # match transmissions strings
        m = regex_tx.match(line)
        if m:

            # get dictionary of matched groups
            d = m.groupdict()

            # increase the count of transmissions from this address
            tx_id = idsmap[d['addr']]
            tx[tx_id] = tx.get(tx_id, 0) + 1

            continue

        # match reception strings
        m = regex_rx.match(line)
        if m:

            # get dictionary of matched groups
            d = m.groupdict()

            # increase the count of receptions for the given link
            from_id = idsmap[d['addr_from']]
            to_id = idsmap[d['addr_to']]
            rx_update = rx.get(from_id, dict())
            rx_update[to_id] = rx_update.get(to_id, 0) + 1
            rx[from_id] = rx_update

            # collect RSSI measurements for the given link
            rssi_update = rssi.get(from_id, dict())
            rssi_update[to_id] = rssi_update.get(to_id, 0) + int(d['rssi'])
            rssi[from_id] = rssi_update

            continue

In [34]:
# diplay results
for tx_id in tx:
    print(f"FROM {tx_id}\t\t{tx[tx_id]}")
    for rx_id in rx[tx_id]:
        rx_from_tx = rx[tx_id][rx_id]
        ppr = math.floor((rx_from_tx / tx[tx_id]) * 100)
        rssi_avg = round(rssi[tx_id][rx_id]/rx_from_tx, 1)
        print(f"\tTO {rx_id}\t{ppr}%\t{rssi_avg}")
    print("\n")

FROM 129		22
	TO 130	21	95%	-87.0
	TO 131	5	22%	-101.4
	TO 132	8	36%	-99.9
	TO 128	4	18%	-100.2


FROM 149		22
	TO 144	20	90%	-82.2
	TO 147	22	100%	-95.9
	TO 153	22	100%	-98.9
	TO 142	17	77%	-94.3
	TO 151	21	95%	-98.3
	TO 150	22	100%	-93.0
	TO 143	20	90%	-89.2
	TO 148	22	100%	-91.0
	TO 145	20	90%	-99.8
	TO 141	1	4%	-98.0


FROM 140		22
	TO 143	19	86%	-84.9
	TO 148	21	95%	-98.8
	TO 145	22	100%	-96.9
	TO 141	22	100%	-78.5
	TO 137	22	100%	-89.5
	TO 138	21	95%	-87.8
	TO 146	18	81%	-99.5
	TO 139	22	100%	-51.0
	TO 144	19	86%	-85.9
	TO 136	21	95%	-93.0
	TO 142	22	100%	-74.0


FROM 28		22
	TO 33	18	81%	-81.2
	TO 24	13	59%	-94.2
	TO 29	21	95%	-81.2
	TO 31	14	63%	-92.4
	TO 25	17	77%	-82.6
	TO 22	15	68%	-89.3
	TO 23	12	54%	-97.0
	TO 26	17	77%	-81.5
	TO 30	19	86%	-88.6
	TO 32	18	81%	-84.5
	TO 27	18	81%	-74.6
	TO 21	10	45%	-95.5
	TO 100	9	40%	-100.0
	TO 35	2	9%	-99.0
	TO 101	3	13%	-101.3
	TO 34	3	13%	-98.0


FROM 111		22
	TO 31	13	59%	-93.8
	TO 152	11	50%	-100.1
	TO 110	20	90%	-90.5
	TO 101	9	40%	-